In [11]:
import time
import torch
from torch import nn,optim
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
def vgg_block(num_convs,in_channels,out_channels):
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1))
        else:
            blk.append(nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1))
        blk.append(nn.ReLU())
    blk.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*blk)

In [13]:
conv_arch = ((1,1,64),(1,64,128),(2,128,256),(2,256,512),(2,512,512))
fc_features = 512*7*7
fc_hidden_units = 4096

In [14]:
def vgg(conv_arch,fc_features,fc_hidden_units=4096):
    net = nn.Sequential()
    for i, (num_convs,in_channels,out_channels) in enumerate(conv_arch):
        net.add_module("vgg_block_"+str(i+1),vgg_block(num_convs,in_channels,out_channels))
    net.add_module("fc",
                  nn.Sequential(d2l.FlattenLayer(),
                               nn.Linear(fc_features,fc_hidden_units),
                               nn.ReLU(),
                               nn.Dropout(0.5),
                               nn.Linear(fc_hidden_units,fc_hidden_units),
                               nn.ReLU(),
                               nn.Dropout(0.5),
                               nn.Linear(fc_hidden_units,10)
                               ))
    return net

In [15]:
net = vgg(conv_arch,fc_features,fc_hidden_units)
X = torch.rand(1,1,224,224)
for name, blk in net.named_children():
    X = blk(X)
    print(name,'output shape:',X.shape)

vgg_block_1 output shape: torch.Size([1, 64, 112, 112])
vgg_block_2 output shape: torch.Size([1, 128, 56, 56])


RuntimeError: Given groups=1, weight of size [256, 128, 3, 3], expected input[1, 256, 56, 56] to have 128 channels, but got 256 channels instead